In [15]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output

import base64
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
import Module5CRUD



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "Emmanuel3!"
shelter = Module5CRUD.AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))




#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Hr(),
    html.Div(id='selection-output-container'),
    html.Div(dcc.Dropdown(options=[{'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                          {'label' : 'Mountain or Wilderness Rescue', 'value' : 'Mountain or Wilderness Rescue'},
                          {'label' : 'Disaster or Individual Tracking Rescue', 'value' : 'Disaster or Individual Tracking Rescue'},
                          {'label' : 'Reset', 'value' : 'Reset'}],
                          placeholder='Select a Filter', id='selection-dropdown')),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # Sets up the features for an interactive data table to make it user-friendly
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    html.Center(html.Div(className='row', style={'display' : 'flex'}, children=[
        html.Div(dcc.Graph(id='pie-graph-id'), className='col s12 m6'),
        html.Div([
            dl.Map(children=[dl.TileLayer()],
            id='map-id',
            className='col s12 m6'
            )])])),
    html.H1("Gabriel Feng SNHU CS340 Final Project")
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('datatable-id', 'data'),
    [Input('selection-dropdown', 'value')]
)
def update_table(value):
    if value == 'Water Rescue':
        df = pd.DataFrame.from_records(shelter.read({'breed' : 
                                                     {'$in' : ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 
                                                               'Newfoundland', 'Labrador Retriever/Newfoundland', 
                                                               'Newfoundland Mix', 'Newfoundland/Labrador Retriever']}, 
                                                     'sex_upon_outcome' : 'Intact Female',
                                                     'age_upon_outcome_in_weeks' : {'$gte': 26, '$lte': 156}}))
    elif value == 'Mountain or Wilderness Rescue':
        df = pd.DataFrame.from_records(shelter.read({'breed' : 
                                                     {'$in' : ['German Shepherd', 'Alaskan Malamute', 
                                                               'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']}, 
                                                     'sex_upon_outcome' : 'Intact Male',
                                                     'age_upon_outcome_in_weeks' : {'$gte': 26, '$lte': 156}}))
    elif value == 'Disaster or Individual Tracking Rescue':
        df = pd.DataFrame.from_records(shelter.read({'breed' : 
                                                     {'$in' : ['Doberman Pinscher', 'German Shepherd', 
                                                               'Golden Retriever', 'Bloodhound', 'Rottweiler']}, 
                                                     'sex_upon_outcome' : 'Intact Male',
                                                     'age_upon_outcome_in_weeks' : {'$gte': 20, '$lte': 300}}))
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
    return df.to_dict('records')

@app.callback(
    Output('pie-graph-id', "figure"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_indices")]
)
def update_graph(viewData, rows):
    viewDf = pd.DataFrame.from_dict(viewData)
    dff = viewDf.loc[rows]
    dff.loc[dff['1'] >= 1, '1'] = 1
    fig = px.pie(dff, values='1', names='breed')
    return fig
     
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', "derived_viewport_indices"),
     Input('datatable-id', "selected_rows")]
)
def update_map(viewData, rows, selectedRow):
#FIXME Add in the code for your geolocation chart
    viewDf = pd.DataFrame.from_dict(viewData)
    dff = viewDf.loc[rows]
    #Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.loc[selectedRow[0], 'location_lat'], dff.loc[selectedRow[0], 'location_long']], children=[
                dl.Tooltip(dff.loc[selectedRow[0], 'breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.loc[selectedRow[0],'name'])
                ])
            ])
        ])
    ]

app

Connected Properly
